In [2]:
import os
from pathlib import Path
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pickle

In [3]:
SAO_BASE_DIR_NAME = 'F:/Ionosonde/SAO_pick'
SAO_BASE_DIR = Path(SAO_BASE_DIR_NAME)
SBF_BASE_DIR_NAME = 'F:/Ionosonde/SBF_mat'
SBF_BASE_DIR = Path(SBF_BASE_DIR_NAME)
SAO_AUTO_DIR_NAME = 'F:/Ionosonde/SAO_pick_ARTIST/'
SAO_AUTO_DIR = Path(SAO_AUTO_DIR_NAME)

In [4]:
Iono_save_dir = 'F:/Ionosonde/Dataset1230/IonogramData/'
ARTISTScale_save_dir = 'F:/Ionosonde/Dataset1230/ARTISTScale/'
ExpertScale_save_dir = 'F:/Ionosonde/Dataset1230/ExpertScale/'

In [10]:
counter = 0
record_file = open('F:/Ionosonde/Dataset1230/data.lst','w')
# save int
gt_x_list = list()
gt_y_list = list()
auto_y_list = list()
for SAOs in SAO_BASE_DIR.glob('*.mat'):
    SAO_day = scipy.io.loadmat(str(SAOs))
    F2O = SAO_day['t_F2O']
    F1O = SAO_day['t_F1O']
    EO = SAO_day['t_EO']
    if len(F1O) < 1:
        continue
    if os.path.exists(SAO_AUTO_DIR_NAME+SAOs.name[:-11]+'.SAO.mat'):
        #print('hit')
        auto_flag = True
    else:
        #print(SAO_AUTO_DIR_NAME+SAOs.name[:-11]+'.SAO.mat')
        auto_flag = False
    
    if auto_flag:
        SAO_day_auto = scipy.io.loadmat(SAO_AUTO_DIR_NAME+SAOs.name[:-11]+'.SAO.mat')
        F2O_auto = SAO_day_auto['t_F2O']
        F1O_auto = SAO_day_auto['t_F1O']
        EO_auto = SAO_day_auto['t_EO']

    for idx in range(len(SAO_day['TS'])-1):
        t_TS = SAO_day['TS'][idx+1]
        timecode = SAO_day['TS'][idx+1][2:6]+SAO_day['TS'][idx+1][9:19]
        t_mat_name = SBF_BASE_DIR_NAME + '/WU430_' + timecode + '.SBF.mat'
        
        if os.path.exists(t_mat_name):
            pass
        else:
            continue
        
        t_mat = scipy.io.loadmat(t_mat_name)

        amp_O = t_mat['AmplitudeO']
        amp_X = t_mat['AmplitudeX']
        MPA_O = t_mat['MPAO']
        MPA_X = t_mat['MPAX']
        Hs = float(t_mat['HeightStart'][0][0])
        Hd = float(t_mat['HeightStep'][0][0])
        Fs = float(t_mat['FrequencyStart'][0][0])
        Fd = float(t_mat['FrequencyStep'][0][0])

        input_array = np.zeros([np.shape(amp_O)[0],np.shape(amp_O)[1],3],dtype='uint8')
        input_array[:,:,0] = amp_O
        input_array[:,:,1] = amp_X

        if np.shape(input_array)[1] != np.shape(MPA_O)[1]:
            for jdx in range(np.shape(amp_O)[0]):
                input_array[jdx,:,2] = MPA_O[0,:np.shape(input_array)[1]]
        else:
            for jdx in range(np.shape(amp_O)[0]):
                input_array[jdx,:,2] = MPA_O

        print(timecode)

        t_mask = np.zeros([np.shape(amp_O)[0],np.shape(amp_O)[1],3],dtype='uint8')

        EO_length = int(len(EO[idx*2+1])/8)

        EO_freq = []
        EO_height = []
        for kdx in range(EO_length):
            #print(F2O[idx*2+1][kdx*8:(kdx+1)*8])
            try:
                if float(EO[idx*2+2][kdx*8:(kdx+1)*8]) > 3000:
                    continue
                EO_freq.append((float(EO[idx*2+1][kdx*8:(kdx+1)*8])-Fs)/Fd)
                EO_height.append((float(EO[idx*2+2][kdx*8:(kdx+1)*8])-Hs)/Hd)
            except:
                break
        EO_length = len(EO_freq)
        """
        if EO_length > 0:
            pass
        else:
            continue
        """
        F1O_length = int(len(F1O[idx*2+1])/8)

        F1O_freq = []
        F1O_height = []
        for kdx in range(F1O_length):
            #print(F2O[idx*2+1][kdx*8:(kdx+1)*8])
            try:
                if float(F1O[idx*2+2][kdx*8:(kdx+1)*8]) > 3000:
                    continue
                F1O_freq.append((float(F1O[idx*2+1][kdx*8:(kdx+1)*8])-Fs)/Fd)
                F1O_height.append((float(F1O[idx*2+2][kdx*8:(kdx+1)*8])-Hs)/Hd)
            except:
                break
        F1O_length = len(F1O_freq)
        """
        if F1O_length > 0:
            pass
        else:
            continue
        """
        F2O_length = int(len(F2O[idx*2+1])/8)
        
        F2O_freq = []
        F2O_height = []
        for kdx in range(F2O_length):
            #print(F2O[idx*2+1][kdx*8:(kdx+1)*8])
            try:
                if float(F2O[idx*2+2][kdx*8:(kdx+1)*8]) > 3000:
                    continue
                F2O_freq.append((float(F2O[idx*2+1][kdx*8:(kdx+1)*8])-Fs)/Fd)
                F2O_height.append((float(F2O[idx*2+2][kdx*8:(kdx+1)*8])-Hs)/Hd)
            except:
                break
        F2O_length = len(F2O_freq)
        """
        if F2O_length > 0:
            pass
        else:
            continue
        """
        for kdx in range(len(EO_freq)-1):
            t_gap = int(EO_freq[kdx+1]) - int(EO_freq[kdx])
            t_k = int(EO_height[kdx+1]) - int(EO_height[kdx])
            t_mask[int(EO_height[kdx]),int(EO_freq[kdx]),0] = 1
            for mdx in range(t_gap):
                t_mask[int(EO_height[kdx]+(mdx+1)*t_k),int(EO_freq[kdx]+mdx+1),0] = 1

        for kdx in range(len(F1O_freq)-1):
            t_gap = int(F1O_freq[kdx+1]) - int(F1O_freq[kdx])
            t_k = int(F1O_height[kdx+1]) - int(F1O_height[kdx])
            t_mask[int(F1O_height[kdx]),int(F1O_freq[kdx]),1] = 1
            for mdx in range(t_gap):
                t_mask[int(F1O_height[kdx])+(mdx+1)*t_k,int(F1O_freq[kdx]+mdx+1),1] = 1


        for kdx in range(len(F2O_freq)-1):
            t_gap = int(F2O_freq[kdx+1]) - int(F2O_freq[kdx])
            t_k = int(F2O_height[kdx+1]) - int(F2O_height[kdx])
            t_mask[int(F2O_height[kdx]),int(F2O_freq[kdx]),2] = 1
            for mdx in range(t_gap):
                t_mask[int(F2O_height[kdx])+(mdx+1)*t_k,int(F2O_freq[kdx]+mdx+1),2] = 1
        
        #auto mask
        if auto_flag:
            auto_mask = np.zeros([np.shape(amp_O)[0],np.shape(amp_O)[1],3],dtype='uint8')
            try:
                EO_length_auto = int(len(EO_auto[idx*2+1])/8)
            except:
                EO_length_auto = 0
            EO_freq = []
            EO_height = []
            for kdx in range(EO_length_auto):
                #print(F2O[idx*2+1][kdx*8:(kdx+1)*8])
                try:
                    if float(EO_auto[idx*2+2][kdx*8:(kdx+1)*8]) > 3000:
                        continue
                    EO_freq.append((float(EO_auto[idx*2+1][kdx*8:(kdx+1)*8])-Fs)/Fd)
                    EO_height.append((float(EO_auto[idx*2+2][kdx*8:(kdx+1)*8])-Hs)/Hd)
                except:
                    break
            EO_length_auto = len(EO_freq)
            """
            if EO_length > 0:
                pass
            else:
                continue
            """
            try:
                F1O_length_auto = int(len(F1O_auto[idx*2+1])/8)
            except:
                F1O_length_auto = 0

            F1O_freq = []
            F1O_height = []
            for kdx in range(F1O_length_auto):
                #print(F2O[idx*2+1][kdx*8:(kdx+1)*8])
                try:
                    if float(F1O_auto[idx*2+2][kdx*8:(kdx+1)*8]) > 3000:
                        continue
                    F1O_freq.append((float(F1O_auto[idx*2+1][kdx*8:(kdx+1)*8])-Fs)/Fd)
                    F1O_height.append((float(F1O_auto[idx*2+2][kdx*8:(kdx+1)*8])-Hs)/Hd)
                except:
                    break
            F1O_length_auto = len(F1O_freq)
            """
            if F1O_length > 0:
                pass
            else:
                continue
            """
            try:
                F2O_length_auto = int(len(F2O_auto[idx*2+1])/8)
            except:
                F2O_length_auto = 0

            F2O_freq = []
            F2O_height = []
            for kdx in range(F2O_length_auto):
                #print(F2O[idx*2+1][kdx*8:(kdx+1)*8])
                try:
                    if float(F2O_auto[idx*2+2][kdx*8:(kdx+1)*8]) > 3000:
                        continue
                    F2O_freq.append((float(F2O_auto[idx*2+1][kdx*8:(kdx+1)*8])-Fs)/Fd)
                    F2O_height.append((float(F2O_auto[idx*2+2][kdx*8:(kdx+1)*8])-Hs)/Hd)
                except:
                    break
            F2O_length_auto = len(F2O_freq)
            """
            if F2O_length > 0:
                pass
            else:
                continue
            """

            for kdx in range(len(EO_freq)):
                auto_mask[int(EO_height[kdx]),int(EO_freq[kdx]),0] = 1.0

            for kdx in range(len(F1O_freq)):
                auto_mask[int(F1O_height[kdx]),int(F1O_freq[kdx]),1] = 1.0

            for kdx in range(len(F2O_freq)):
                try:
                    auto_mask[int(F2O_height[kdx]),int(F2O_freq[kdx]),2] = 1.0       
                except:
                    continue
        t_f = open(Iono_save_dir+'WU430_' + timecode + '_data.pickle','wb')
        pickle.dump(input_array,t_f)
        t_f.close()
        t_f = open(ExpertScale_save_dir+'WU430_' + timecode + '_Expert.pickle','wb')
        pickle.dump(t_mask,t_f)
        t_f.close()
        if auto_flag:
            t_f = open(ARTISTScale_save_dir+'WU430_' + timecode + '_ARTIST.pickle','wb')
            pickle.dump(auto_mask,t_f)
            t_f.close()
            record_file.write('/IonogramData/WU430_'+timecode+'_data.pickle '+
                             '/ExpertScale/WU430_'+timecode+'_Expert.pickle '+
                             '/ARTISTScale/WU430_'+timecode+'_ARTIST.pickle\n')
        else:
            print('No ARTIST...Skipping...')
            #record_file.write(timecode + ' noAuto\n')
        #break
record_file.close()

20180201051500
20180201053000
20180201054500
20180201060000
20180201061500
20180201063000
20180201064500
20180201070000
20180201071500
20180201073000
20180201074500
20180201080000
20180201081500
20180201083000
20180201084500
20180201090000
20180201091500
20180201093000
20180201094500
20180201100000
20180201101500
20180201103000
20180201104500
20180201110000
20180201111500
20180201113000
20180201114500
20180201120000
20180201121500
20180201123000
20180201124500
20180201130000
20180201131500
20180201133000
20180201134500
20180201140000
20180201141500
20180201143000
20180201144500
20180201150000
20180201151500
20180201153000
20180201154500
20180201160000
20180201161500
20180201163000
20180201164500
20180201170000
20180201171500
20180201173000
20180201174500
20180201180000
20180201181500
20180201183000
20180201184500
20180201190000
20180201191500
20180201193000
20180201194500
20180201200000
20180201201500
20180201203000
20180201204500
20180201210000
20180201211500
20180201213000
2018020121

20180206220000
20180206221500
20180206223000
20180206224500
20180206230000
20180206231500
20180206233000
20180206234500
20180207000000
20180207001500
20180207003000
20180207004500
20180207010000
20180207011500
20180207013000
20180207014500
20180207020000
20180207021500
20180207023000
20180207024500
20180207030000
20180207031500
20180207033000
20180207034500
20180207040000
20180207041500
20180207043000
20180207044500
20180207050000
20180207051500
20180207053000
20180207054500
20180207060000
20180207061500
20180207063000
20180207064500
20180207070000
20180207071500
20180207073000
20180207074500
20180207080000
20180207081500
20180207083000
20180207084500
20180207090000
20180207091500
20180207093000
20180207094500
20180207100000
20180207101500
20180207103000
20180207104500
20180207110000
20180207111500
20180207113000
20180207114500
20180207120000
20180207121500
20180207123000
20180207124500
20180207130000
20180207131500
20180207133000
20180207134500
20180207140000
20180207141500
2018020714

20180212144500
20180212150000
20180212151500
20180212153000
20180212154500
20180212160000
20180212161500
20180212163000
20180212164500
20180212170000
20180212171500
20180212173000
20180212174500
20180212180000
20180212181500
20180212183000
20180212184500
20180212190000
20180212191500
20180212193000
20180212194500
20180212200000
20180212201500
20180212203000
20180212204500
20180212210000
20180212211500
20180212213000
20180212214500
20180212220000
20180212221500
20180212223000
20180212224500
20180212230000
20180212231500
20180212233000
20180212234500
20180213000000
20180213001500
20180213003000
20180213004500
20180213010000
20180213011500
20180213013000
20180213014500
20180213020000
20180213021500
20180213023000
20180213024500
20180213030000
20180213031500
20180213033000
20180213034500
20180213040000
20180213041500
20180213043000
20180213044500
20180213050000
20180213051500
20180213053000
20180213054500
20180213060000
20180213061500
20180213063000
20180213064500
20180213070000
2018021307

20180218073000
20180218074500
20180218080000
20180218081500
20180218083000
20180218084500
20180218090000
20180218091500
20180218093000
20180218094500
20180218100000
20180218101500
20180218103000
20180218104500
20180218110000
20180218111500
20180218113000
20180218114500
20180218120000
20180218121500
20180218123000
20180218124500
20180218130000
20180218131500
20180218133000
20180218134500
20180218140000
20180218141500
20180218143000
20180218144500
20180218150000
20180218151500
20180218153000
20180218154500
20180218160000
20180218161500
20180218163000
20180218164500
20180218170000
20180218171500
20180218173000
20180218174500
20180218180000
20180218181500
20180218183000
20180218184500
20180218190000
20180218191500
20180218193000
20180218194500
20180218200000
20180218201500
20180218203000
20180218204500
20180218210000
20180218211500
20180218213000
20180218214500
20180218220000
20180218221500
20180218223000
20180218224500
20180218230000
20180218231500
20180218233000
20180218234500
2018021900

20180224001500
20180224003000
20180224004500
20180224010000
20180224011500
20180224013000
20180224014500
20180224020000
20180224021500
20180224023000
20180224024500
20180224030000
20180224031500
20180224033000
20180224034500
20180224040000
20180224041500
20180224043000
20180224044500
20180224050000
20180224051500
20180224053000
20180224054500
20180224060000
20180224061500
20180224063000
20180224064500
20180224070000
20180224071500
20180224073000
20180224074500
20180224080000
20180224081500
20180224083000
20180224084500
20180224090000
20180224091500
20180224093000
20180224094500
20180224100000
20180224101500
20180224103000
20180224104500
20180224110000
20180224111500
20180224113000
20180224114500
20180224120000
20180224121500
20180224123000
20180224124500
20180224130000
20180224131500
20180224133000
20180224134500
20180224140000
20180224141500
20180224143000
20180224144500
20180224150000
20180224151500
20180224153000
20180224154500
20180224160000
20180224161500
20180224163000
2018022416

...

20180806233000
20180806234500
20180807000000
20180807001500
20180807003000
20180807004500
20180807010000
20180807011500
20180807013000
20180807014500
20180807020000
20180807021500
20180807023000
20180807024500
20180807030000
20180807031500
20180807033000
20180807034500
20180807040000
20180807041500
20180807043000
20180807044500
20180807050000
20180807051500
20180807053000
20180807054500
20180807060000
20180807061500
20180807063000
20180807064500
20180807070000
20180807071500
20180807073000
20180807074500
20180807080000
20180807081500
20180807083000
20180807084500
20180807090000
20180807091500
20180807093000
20180807094500
20180807100000
20180807101500
20180807103000
20180807104500
20180807110000
20180807111500
20180807113000
20180807114500
20180807120000
20180807121500
20180807123000
20180807124500
20180807130000
20180807131500
20180807133000
20180807134500
20180807140000
20180807141500
20180807143000
20180807144500
20180807150000
20180807151500
20180807153000
20180807154500
2018080716

20180812161500
20180812163000
20180812164500
20180812170000
20180812171500
20180812173000
20180812174500
20180812180000
20180812181500
20180812183000
20180812184500
20180812190000
20180812191500
20180812193000
20180812194500
20180812200000
20180812201500
20180812203000
20180812204500
20180812210000
20180812211500
20180812213000
20180812214500
20180812220000
20180812221500
20180812223000
20180812224500
20180812230000
20180812231500
20180812233000
20180812234500
20180813000000
20180813001500
20180813003000
20180813004500
20180813010000
20180813011500
20180813013000
20180813014500
20180813020000
20180813021500
20180813023000
20180813024500
20180813030000
20180813031500
20180813033000
20180813034500
20180813040000
20180813041500
20180813043000
20180813044500
20180813050000
20180813051500
20180813053000
20180813054500
20180813060000
20180813061500
20180813063000
20180813064500
20180813070000
20180813071500
20180813073000
20180813074500
20180813080000
20180813081500
20180813083000
2018081308

20180818091500
20180818093000
20180818094500
20180818100000
20180818101500
20180818103000
20180818104500
20180818110000
20180818111500
20180818113000
20180818114500
20180818120000
20180818121500
20180818123000
20180818124500
20180818130000
20180818131500
20180818133000
20180818134500
20180818140000
20180818141500
20180818143000
20180818144500
20180818150000
20180818151500
20180818153000
20180818154500
20180818160000
20180818161500
20180818163000
20180818164500
20180818170000
20180818171500
20180818173000
20180818174500
20180818180000
20180818181500
20180818183000
20180818184500
20180818190000
20180818191500
20180818193000
20180818194500
20180818200000
20180818201500
20180818203000
20180818204500
20180818210000
20180818211500
20180818213000
20180818214500
20180818220000
20180818221500
20180818223000
20180818224500
20180818230000
20180818231500
20180818233000
20180818234500
20180819000000
20180819001500
20180819003000
20180819004500
20180819010000
20180819011500
20180819013000
2018081901

20180824020000
20180824021500
20180824023000
20180824024500
20180824030000
20180824031500
20180824033000
20180824034500
20180824040000
20180824041500
20180824043000
20180824044500
20180824050000
20180824051500
20180824053000
20180824054500
20180824060000
20180824061500
20180824063000
20180824064500
20180824070000
20180824071500
20180824073000
20180824074500
20180824080000
20180824081500
20180824083000
20180824084500
20180824090000
20180824091500
20180824093000
20180824094500
20180824100000
20180824101500
20180824103000
20180824104500
20180824110000
20180824111500
20180824113000
20180824114500
20180824120000
20180824121500
20180824123000
20180824124500
20180824130000
20180824131500
20180824133000
20180824134500
20180824140000
20180824141500
20180824143000
20180824144500
20180824150000
20180824151500
20180824153000
20180824154500
20180824160000
20180824161500
20180824163000
20180824164500
20180824170000
20180824171500
20180824173000
20180824174500
20180824180000
20180824181500
2018082418

20180829190000
20180829191500
20180829193000
20180829194500
20180829200000
20180829201500
20180829203000
20180829204500
20180829210000
20180829211500
20180829213000
20180829214500
20180829220000
20180829221500
20180829223000
20180829224500
20180829230000
20180829231500
20180829233000
20180829234500
20180830000000
20180830001500
20180830003000
20180830004500
20180830010000
20180830011500
20180830013000
20180830014500
20180830020000
20180830021500
20180830023000
20180830024500
20180830030000
20180830031500
20180830033000
20180830034500
20180830040000
20180830041500
20180830043000
20180830044500
20180830050000
20180830051500
20180830053000
20180830054500
20180830060000
20180830061500
20180830063000
20180830064500
20180830070000
20180830071500
20180830073000
20180830074500
20180830080000
20180830081500
20180830083000
20180830084500
20180830090000
20180830091500
20180830093000
20180830094500
20180830100000
20180830101500
20180830103000
20180830104500
20180830110000
20180830111500
2018083011

In [9]:
record_file.close()

In [ ]:
test_read_file = open('','r')